In [ ]:
Messages = []
Messages.append("Please enter your bank account number here ")
Messages.append("Winner claim secret prize now")
Messages.append("You’ve Won!")
Messages.append(" How was your day")
Messages.append("Amazon Gift card worth Rs 500!!! ")
Messages.append("Winner claim secret prize now")
Messages.append("Hurry !Claim your prize of Rs 50000 now")
Messages.append("Good luck with School")
Messages.append("Buy our new product to give you guaranteed results")
Messages.append("Let's go shopping tomorrow!")
Mes = input("Enter message ")
Messages.append(Mes)



Enter message send credit card info 


In [ ]:
import collections
import numpy as np
import csv

In [ ]:
def fetch_words(message):
    """Get the list of words from a message .

     
    """
    #We remove punctuations in the message
    punctuations='~`!@#$%^&*(),.:\";\'-+=_0123456789?£'
    for p in punctuations:
        message = message.replace(p,'')
    words =  message.lower().split(' ')
    return words
  

In [ ]:
def create_dictionary(messages):
    """ Function to create a dictionary out of the cleaned list of words. 
    We only select words that occur more than 5 times in all of the Data Corpus. 
    The words are mapped to their indices.     
    """
    counts=dict()
    for msgs in messages:

        words = fetch_words(msgs)

        #remove duplicates
        words = list(dict.fromkeys(words))

        #count words
        for word in words:
            counts[word] = counts.get(word,0)+1

    my_dict={}
    my_list=[]

    #consider only keys where count >=5
    for key in counts.keys():
        if counts[key]>=5:
            my_list.append(key)

    my_list.sort()    
    for i in range (0,len(my_list)):
        my_dict[my_list[i]]=i
    return my_dict
   


In [ ]:
def transform_text(messages, word_dictionary):
    """Transform a list of messages into a numpy array.

    This function creates a numpy array that contains the number of times each word
    of the vocabulary appears in each message. 
    Each row in the resulting array corresponds to each message 
    and each column corresponds to a word of the vocabulary.
    """
    vocab =  sorted(word_dictionary.keys())
    vocab_length = len(word_dictionary.keys())
    num_messages = len(messages)

    arr = np.zeros((num_messages,vocab_length), dtype=np.float128)
    for i in range(0,num_messages):
        words=fetch_words(messages[i])
          
        for word in words:
            if word in vocab:
                j = vocab.index(word)
                arr[i][j]+=1

    return arr
  


In [ ]:

def fit_naive_bayes_model(matrix, labels):
    #We then create our model using the Bayesian formula for conditional probability. 
    """Fit a naive bayes model.

    This function fits a Naive Bayes model when given a training matrix and labels.
    """
    ph_y1_arr = []
    ph_y0_arr = []
    ph_y = 0
    
    # ph_y is overall probability that a message is spam.
    num_messages,vocab = matrix.shape
    #Split into Spam and non spam matrices
    for i in range(0,num_messages):
        if labels[i]==1:
            ph_y1_arr.append(matrix[i])
        else:
            ph_y0_arr.append(matrix[i])
    
    #convert lists to numpy arrays
    ph_y1_np = np.array(ph_y1_arr)
    ph_y0_np = np.array(ph_y0_arr)

    ph_y1 = (1+np.sum(ph_y1_np, axis=0))/(len(ph_y1_np)+vocab)
    ph_y0 = (1+np.sum(ph_y0_np, axis=0))/(len(ph_y0_np)+vocab)

    ph_y= len(ph_y1_np)/num_messages

    return ph_y0, ph_y1, ph_y
    

In [ ]:
def predict_from_naive_bayes_model(model, matrix):
    #Use a Naive Bayes model to compute predictions for a target matrix.
    ph_y0, ph_y1, ph_y = model

    matrix[matrix>1]=1

    y1 = ph_y1*matrix
    y1[y1==0]=1 
    y0 = ph_y0*matrix
    y0[y0==0]=1 
    
    num_messages, vocab = matrix.shape

    p1= np.exp(np.sum(np.log(y1), axis=1))*ph_y
    

    p0= np.exp(np.sum(np.log(y0), axis=1))*(1-ph_y)
   
    probabilities=[]
    for i in range(0,len(p0)):
        if p0[i]>=p1[i]:
            probabilities.append(0)
        else:
            probabilities.append(1)
    return np.array(probabilities)

In [ ]:
def fetch_top_five_naive_bayes_words(model, dictionary):
    """ Here we compute the top five words that are most indicative of
    the spam (i.e positive) class.
    """

    reversed_dictionary = {v : k for (k, v) in dictionary.items()}
    
    phi_y0,phi_y1,phi_y = model

    logs = np.log(phi_y1)-np.log(phi_y0) 
    sorted_args = np.argsort(logs)
    top_5 = sorted_args[-5:]    
    ret = [reversed_dictionary[i] for i in top_5]

    return ret

In [ ]:
def load_spam_dataset(tsv_path):
    messages = []
    labels = []

    with open(tsv_path, 'r', newline='', encoding='utf8') as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t', quoting=csv.QUOTE_NONE)

        for label, message in reader:
            messages.append(message)
            labels.append(1 if label == 'spam' else 0)

    return messages, np.array(labels)

In [ ]:
def main():
    train_messages, train_labels = load_spam_dataset('spam_train.tsv')
    val_messages, val_labels = load_spam_dataset('spam_val.tsv')
    test_messages, test_labels = load_spam_dataset('spam_test.tsv')

    dictionary = create_dictionary(train_messages)

    print('Size of dictionary: ', len(dictionary))

  
    train_matrix = transform_text(train_messages, dictionary)

    
    val_matrix = transform_text(val_messages, dictionary)
    test_matrix = transform_text(test_messages, dictionary)

    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

    naive_bayes_predictions_v = predict_from_naive_bayes_model(naive_bayes_model, val_matrix)


    naive_bayes_accuracy_v = np.mean(naive_bayes_predictions_v == val_labels)

    print('\nNaive Bayes had an accuracy of {} on the validation set'.format(naive_bayes_accuracy_v))

    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

    print('\nNaive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

    top_5_words = fetch_top_five_naive_bayes_words(naive_bayes_model, dictionary)

    print('\nThe top 5 indicative words for Naive Bayes are: ', top_5_words)


    msg_matrix = transform_text(Messages, dictionary)
    spam_or_ham = predict_from_naive_bayes_model(naive_bayes_model, msg_matrix)

    for i in range(len(Messages)):
      if spam_or_ham[i]==1:
        print("\nThe message \"{}\" is:\nSPAM!".format(Messages[i]))
      else: 
        print("\nThe message \"{}\" is:\nNOT SPAM!".format(Messages[i]))

if __name__ == "__main__":
    main()

Size of dictionary:  1498

Naive Bayes had an accuracy of 0.9820466786355476 on the validation set

Naive Bayes had an accuracy of 0.9874551971326165 on the testing set

The top 5 indicative words for Naive Bayes are:  ['guaranteed', 'tone', 'won', 'prize', 'claim']

The message "Please enter your bank account number here " is:
SPAM!

The message "Winner claim secret prize now" is:
SPAM!

The message "You’ve Won!" is:
SPAM!

The message " How was your day" is:
NOT SPAM!

The message "Amazon Gift card worth Rs 500!!! " is:
SPAM!

The message "Winner claim secret prize now" is:
SPAM!

The message "Hurry !Claim your prize of Rs 50000 now" is:
SPAM!

The message "Good luck with School" is:
NOT SPAM!

The message "Buy our new product to give you guaranteed results" is:
SPAM!

The message "Let's go shopping tomorrow!" is:
NOT SPAM!

The message "send credit card info " is:
SPAM!
